In [5]:
import pyspark
from pyspark.sql import SparkSession
sc = pyspark.SparkContext('local[*]')
spark = SparkSession.builder.appName("csvTest").getOrCreate()

In [11]:
global Path
if sc.master[0:5] == "local":
    Path = "file:/home/jovyan/work/csvData/"
else:
    Path = "hdfs:/user/zeppelin/csvData/"

In [58]:
from operator import add

In [7]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [12]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("product", StringType(), True),
    StructField("kind", StringType(), True),
    StructField("category", StringType(), True),
    StructField("price", IntegerType(), True),
    StructField("url", StringType(), True),
    StructField("website", StringType(), True),
    StructField("picture", StringType(), True),
    StructField("update_time", DateType(), True)])

df = spark.read.csv(Path+"test.csv",header=False,schema=schema)

In [35]:
df.show(5)

+---+--------------------+----+--------+-----+--------------------+-------+--------------------+-----------+
| id|             product|kind|category|price|                 url|website|             picture|update_time|
+---+--------------------+----+--------+-----+--------------------+-------+--------------------+-----------+
|  1|纯木竹浆卫生纸本色纸抽纸面巾妇婴餐...| 衛生紙|    日常用品|    1|//item.taobao.com...|     淘寶|//g-search3.alicd...| 2017-07-17|
|  2|包邮 百慧2800克实惠装家用无芯...| 衛生紙|    日常用品|    1|//item.taobao.com...|     淘寶|//g-search2.alicd...| 2017-07-17|
|  3|理文 本色纸抽纸原生竹浆纤维纸婴儿...| 衛生紙|    日常用品|    1|//item.taobao.com...|     淘寶|//g-search3.alicd...| 2017-07-17|
|  4|女主角卫生纸卷筒纸原生浆纸家用厕纸...| 衛生紙|    日常用品|    1|//item.taobao.com...|     淘寶|//g-search2.alicd...| 2017-07-17|
|  5|家庭装24包抽纸整箱厕纸面巾纸卫生...| 衛生紙|    日常用品|    1|//item.taobao.com...|     淘寶|//g-search3.alicd...| 2017-07-17|
+---+--------------------+----+--------+-----+--------------------+-------+--------------------+-----------+
only showing top 5 

In [13]:
bagdf = df.filter("kind='背包' and website='Y拍'")
bagdf.show(5)

+-----+--------------------+----+--------+-----+--------------------+-------+--------------------+-----------+
|   id|             product|kind|category|price|                 url|website|             picture|update_time|
+-----+--------------------+----+--------+-----+--------------------+-------+--------------------+-----------+
|13453|【限時免運 可充電防水 背包 】書...|  背包|    外出用品|  565|https://tw.bid.ya...|     Y拍|//nevec-img.zenfs...| 2017-07-17|
|13454|{日差} 高端STAMPD MCM...|  背包|    外出用品| 1680|https://tw.bid.ya...|     Y拍|//nevec-img.zenfs...| 2017-07-17|
|13455|《生活專科》空運直送正品✈日本樂天...|  背包|    外出用品|  745|https://tw.bid.ya...|     Y拍|//nevec-img.zenfs...| 2017-07-17|
|13456|【Matchwood直營】Matc...|  背包|    外出用品| 1780|https://tw.bid.ya...|     Y拍|//nevec-img.zenfs...| 2017-07-17|
|13457|英國 JUST Hype 雙肩包 ...|  背包|    外出用品|  750|https://tw.bid.ya...|     Y拍|//nevec-img.zenfs...| 2017-07-17|
+-----+--------------------+----+--------+-----+--------------------+-------+--------------------+-----------+
o

In [30]:
bagdf.groupBy('category').count().show()

+--------+-----+
|category|count|
+--------+-----+
|    外出用品|   60|
+--------+-----+



In [37]:
bagdf.select("product").rdd.distinct().count()

60

In [39]:
bagPriceRdd = bagdf.select("price").rdd
bagPriceRdd.take(5)

[Row(price=565),
 Row(price=1680),
 Row(price=745),
 Row(price=1780),
 Row(price=750)]

In [41]:
bagPriceCountRdd = bagPriceRdd.map(lambda x: (x['price'], 1))
bagPriceCountRdd.reduceByKey(lambda x,y: x+y).collect()

[(565, 1),
 (1680, 3),
 (745, 1),
 (1780, 1),
 (750, 2),
 (599, 1),
 (780, 2),
 (790, 3),
 (2980, 2),
 (350, 2),
 (450, 2),
 (999, 1),
 (439, 1),
 (980, 1),
 (490, 2),
 (288, 1),
 (70, 1),
 (1280, 2),
 (109, 1),
 (139, 1),
 (140, 1),
 (900, 1),
 (180, 1),
 (420, 1),
 (300, 1),
 (1150, 1),
 (1880, 1),
 (990, 1),
 (199, 1),
 (1299, 1),
 (1199, 1),
 (1899, 1),
 (799, 1),
 (2180, 1),
 (880, 1),
 (2800, 1),
 (299, 1),
 (579, 1),
 (1980, 1),
 (1480, 1),
 (1790, 1),
 (8400, 1),
 (6200, 1),
 (7300, 1),
 (7001, 1),
 (7800, 1),
 (5600, 1),
 (3050, 1),
 (4050, 1)]

In [52]:
def priceRange(x):
    if 0<=x<1000:
        xrange = "未滿1000"
    elif 1000<=x<3000:
        xrange = "1000~3000"
    elif 3000<=x<5000:
        xrange = "3000~5000"
    elif 5000<x:
        xrange = "5000以上"
    else:
        xrange = str(x)
        
    return xrange

In [57]:
bagPriceRangeRDD = bagPriceCountRdd.map(lambda x: (priceRange(x[0]), x[1]))
bagPriceRangeRDD.reduceByKey(lambda x,y: x+y).collect()

[('未滿1000', 34), ('1000~3000', 18), ('5000以上', 6), ('3000~5000', 2)]

In [59]:
bagPriceMax = bagPriceCountRdd.max(lambda x: x[0])
bagPriceMax

(8400, 1)

In [60]:
bagPriceMin = bagPriceCountRdd.min(lambda x: x[0])
bagPriceMin

(70, 1)

In [68]:
total = sc.accumulator(0.0)
num = sc.accumulator(0)
bagPriceCountRdd.map(lambda x: x[0]).foreach(lambda I: [total.add(I), num.add(1)])
bagPriceAvg = total.value / num.value
bagPriceAvg

1683.7833333333333